In [2]:
import pandas_gbq as gbq
from scipy import stats
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
import plotly.express as px
import numpy as np

dataset_store ="bigquery-public-data.google_analytics_sample.ga_sessions_*"
PROJECT_ID="testebrius"
REGION = "US"

### Queries SQL para extração de dados de produtos

Queries em SQL para extração de dados de produtos de um banco de dados de uma loja virtual e das informações geográficas dos clientes que acessaram o site da loja.

Para identificar a área da loja, assumi que a categoria do produto fornece essa informação.
Além disso, para os produtos, não considerei transações nulas.


In [39]:
# Consulta SQL - Vendas por Produto, Categoria, Receita, Dispositivo e Transações
sql_product = f"""
SELECT
    product.v2ProductName AS product_name,
    product.v2ProductCategory AS product_category,
    product.productPrice AS product_price,
    product.productRevenue AS product_revenue,
    trafficSource.source AS traffic_source,
    trafficSource.medium AS traffic_medium,
    trafficSource.campaign AS traffic_campaign,
    device.deviceCategory AS device_category,
    device.operatingSystem AS device_os,
    device.browser AS device_browser,
    device.isMobile AS device_is_mobile,
    hits.transaction.transactionRevenue AS transaction_revenue
FROM
    `{dataset_store}`, UNNEST(hits) AS hits, UNNEST(hits.product) AS product
WHERE
    hits.transaction.transactionRevenue IS NOT NULL    
;
"""

# Consulta SQL - Vendas por País, Cidade e Dispositivo
sql_map = f"""
SELECT 
  geoNetwork.country AS country,
  geoNetwork.city AS city,
  device.deviceCategory AS device_category,  
  COUNT(*) AS total_accesses,
  SUM(IF(hits.transaction.transactionRevenue IS NOT NULL, 1, 0)) AS total_purchases
FROM `{dataset_store}`, UNNEST(hits) AS hits
GROUP BY country, city, device_category
ORDER BY total_accesses DESC;
"""

### Extração dos dados do BigQuery e criação dos dataframes

Além de extrair os dados dos produtos, aqui também é criado o mapa de calor para visualização dos usuários que acessaram o site e realizararam compras. 

Esse método é bastante demorado, então é recomendado que essa etapa seja pulada e o arquivo salvo em disco country_df.csv seja carregado.

In [ ]:
df_products = gbq.read_gbq(sql_product, project_id=PROJECT_ID)
country_df = gbq.read_gbq(sql_map, project_id=PROJECT_ID)

#verifica se o html do mapa já foi criado - se não, cria
# Filtrando dados para plotar no mapa (usaremos apenas os dados de cidades conhecidas)
country_df = country_df[country_df['city'] != 'not available in demo dataset']

# Adicionar uma coluna com latitudes e longitudes das cidades usando a biblioteca geopy.Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

def get_lat_lon(city, country):
    try:
        location = geolocator.geocode(f"{city}, {country}")
        return location.latitude, location.longitude
    except:
        return None, None

country_df['latitude'], country_df['longitude'] = zip(*country_df.apply(lambda row: get_lat_lon(row['city'], row['country']), axis=1))

# Filtrar as linhas onde a localização é válida
country_df = country_df.dropna(subset=['latitude', 'longitude'])

# Criar o mapa com folium
map_access = folium.Map(location=[0, 0], zoom_start=2)

# Adicionar os pontos de calor
heat_data = [[row['latitude'], row['longitude'], row['total_accesses']] for index, row in country_df.iterrows()]
HeatMap(heat_data).add_to(map_access)

# Salvar o mapa em um arquivo HTML
map_access.save('mapa_acessos.html')

# Ajustar valores de moedas
df_products['product_price'] = df_products['product_price'] / 1000000
df_products['product_revenue'] = df_products['product_revenue'] / 1000000
df_products['transaction_revenue'] = df_products['transaction_revenue'] / 1000000

### Carregamento dos dataframes do disco

Carregamento dos dataframes do disco para evitar a execução do código anterior. 
O dataframe de clientes é carregado do arquivo country_df.csv e já se encontra tratado com as informações de latitude e longitude.

In [3]:
import pandas as pd
df_products = pd.read_csv('df_products.csv')
country_df = pd.read_csv('country_df.csv')

### Análise exploratóiria para identificar missing values e outliers

Análise exploratória dos dados para identificar potenciais nulls, outliers e inconsistências nos dados. Isso foi realizado através de queries no dataframe que filtraram e agruparam a fim de indentificar potenciais nulos ou valores fora do esperado. 

Um exemplo está na coluna de product_category, que apresenta diversos (not set) e um caso de erro de inserção de dados, que salvou uma categoria como ${productitem.product.origCatName}. Optei por substituir os valores faltantes nas colunas e os outliers por "não especificado" ao montar o dashboard, dessa forma se tem uma perda mínima de dados e a visualização se torna mais clara.


In [8]:
df_products['product_category'].unique()

array(['Apparel', 'Drinkware', 'Google', 'Office', 'Bags', 'Headgear',
       'Electronics', 'Lifestyle', '${productitem.product.origCatName}',
       'Accessories', 'Bottles', 'Gift Cards', 'Waze', 'More Bags',
       'Backpacks', 'Notebooks & Journals', '(not set)', 'Housewares',
       'Fun', 'Android', 'Tumblers'], dtype=object)

In [4]:
#give the percentage of occurences of (not set) in each column of the dataframe
df_products.isin(['(not set)']).mean()

product_name           0.000000
product_category       0.218702
product_price          0.000000
product_revenue        0.000000
traffic_source         0.000000
traffic_medium         0.000027
traffic_campaign       0.978092
device_category        0.000000
device_os              0.000000
device_browser         0.000000
device_is_mobile       0.000000
transaction_revenue    0.000000
dtype: float64

In [10]:
#give the percentage of occurences of ${productitem.product.origCatName} in each column of the dataframe
df_products.isin(['${productitem.product.origCatName}']).mean()

product_name           0.000000
product_category       0.007566
product_price          0.000000
product_revenue        0.000000
traffic_source         0.000000
traffic_medium         0.000000
traffic_campaign       0.000000
device_category        0.000000
device_os              0.000000
device_browser         0.000000
device_is_mobile       0.000000
transaction_revenue    0.000000
dtype: float64

In [11]:
# look for missing values
df_products.isnull().mean()

product_name           0.0
product_category       0.0
product_price          0.0
product_revenue        0.0
traffic_source         0.0
traffic_medium         0.0
traffic_campaign       0.0
device_category        0.0
device_os              0.0
device_browser         0.0
device_is_mobile       0.0
transaction_revenue    0.0
dtype: float64

In [12]:
#look for missing values
country_df.isnull().mean()

Unnamed: 0         0.0
country            0.0
city               0.0
device_category    0.0
total_accesses     0.0
total_purchases    0.0
latitude           0.0
longitude          0.0
dtype: float64

In [13]:
#give the percentage of occurences of (not set) in each column of the country_df dataframe
country_df.isin(['(not set)']).mean()

Unnamed: 0         0.000000
country            0.000000
city               0.003947
device_category    0.000000
total_accesses     0.000000
total_purchases    0.000000
latitude           0.000000
longitude          0.000000
dtype: float64

In [14]:
# Substituir valores ausentes por 'não especificado'
df_products['product_category'] = df_products['product_category'].replace('(not set)', 'não especificado')
df_products['product_category'] = df_products['product_category'].replace('${productitem.product.origCatName}', 'não especificado')
df_products['traffic_source'] = df_products['traffic_source'].replace('(not set)', 'não especificado')
df_products['traffic_campaign'] = df_products['traffic_campaign'].replace('(not set)', 'não especificado')
df_products['traffic_medium'] = df_products['traffic_medium'].replace('(not set)', 'não especificado')

In [15]:
# remover os valores ausentes de cidade, uma vez que são poucos
country_df = country_df[country_df['city'] != '(not set)']

#### Análises estatísticas e conclusões iniciais que podem ser extraídas dos dados

Foram utilzadas Análise de Variância (ANOVA) e Teste Shapiro-Wilk para verificar a normalidade dos dados e a presença de outliers

In [30]:
import warnings
warnings.filterwarnings('ignore')

area_sales = df_products.groupby('product_category')['product_revenue'].sum().reset_index()
area_sales = area_sales.sort_values(by='product_revenue', ascending=False)

product_sales = df_products.groupby('product_name')['product_revenue'].sum().reset_index()
product_sales = product_sales.sort_values(by='product_revenue', ascending=False)

# Teste ANOVA para comparar as receitas entre categorias
anova_result_receita = stats.f_oneway(*[df_products[df_products['product_category'] == cat]['product_revenue'] for cat in df_products['product_category'].unique()])
print()
print(f"Resultados ANOVA para comparação de receitas entre categorias de produtos: {anova_result_receita.statistic:.6f} ,  com p-value: {anova_result_receita.pvalue}")

# Teste ANOVA sobre os preços dos produtos
anova_result_price = stats.f_oneway(*[df_products[df_products['product_category'] == cat]['product_price'] for cat in df_products['product_category'].unique()])

print(f"Resultados ANOVA para comparação de preços entre categorias de produtos: {anova_result_price.statistic:.6f} ,  com p-value: {anova_result_price.pvalue}")


shapiro_result_receita = stats.shapiro(df_products['product_revenue'])
print(f"Teste Shapiro-Wilk test para distribuição de receita de produtos: {shapiro_result_receita.statistic:.6f} ,  com p-value: {shapiro_result_receita.pvalue}")

# Conclusões
print("\nExiste alguma área da loja que tem mais vendas que as outras?\n")
top_area = area_sales.iloc[0]
print(f"A área da loja com mais vendas é: {top_area['product_category']} com receita total de {top_area['product_revenue']:.2f}")

print("\nExiste algum produto que tenha destaque?\n")
top_product = product_sales.iloc[0]
print(f"O produto com mais destaque é: {top_product['product_name']} com receita total de {top_product['product_revenue']:.2f}")

print("\nAlguma distribuição possui alguma informação relevante?\n")

if shapiro_result_receita.pvalue < 0.05:
    print("A distribuição das receitas dos produtos não é normal.")
else:
    print("A distribuição das receitas dos produtos é normal.")

if anova_result_receita.pvalue < 0.05:
    print("Há diferenças significativas entre as receitas das diferentes áreas da loja.")
else:
    print("Não há diferenças significativas entre as receitas das diferentes áreas da loja.")
    
if anova_result_price.pvalue < 0.05:
    print("Há diferenças significativas entre os preços dos produtos das diferentes áreas da loja.")
else:
    print("Não há diferenças significativas entre os preços dos produtos das diferentes áreas da loja.")


Resultados ANOVA para comparação de receitas entre categorias de produtos: 27.781257 ,  com p-value: 4.617863859987335e-99
Resultados ANOVA para comparação de preços entre categorias de produtos: 508.274940 ,  com p-value: 0.0
Teste Shapiro-Wilk test para distribuição de receita de produtos: 0.180389 ,  com p-value: 3.820136839163841e-148

Existe alguma área da loja que tem mais vendas que as outras?

A área da loja com mais vendas é: Apparel com receita total de 517059920079.00

Existe algum produto que tenha destaque?

O produto com mais destaque é: Google Men's  Zip Hoodie com receita total de 47636271605.00

Alguma distribuição possui alguma informação relevante?

A distribuição das receitas dos produtos não é normal.
Há diferenças significativas entre as receitas das diferentes áreas da loja.
Há diferenças significativas entre os preços dos produtos das diferentes áreas da loja.


### Dashboard de visualização de dados

Esse dashboard apresenta uma visualização intuitiva e de fácil manipulação das distribuições, rankings e comparações dos produtos, além de um mapa-mundi contendo a distribuição dos usuários que acessaram o site por cidade, permitindo filtrar pelos usuários que realizaram compras.

Esta visualização é essencial para identificar regiões com maior potencial de crescimento e direcionar campanhas de marketing de forma mais eficaz. A visualização em forma de mapa de calor foi escolhida para mostrar a densidade de clientes em cada região de forma mais evidente.

Também é possível avaliar a receita de acordo com o tipo de dispositivo, Sistema Operacional e navegador utilzado pelos clientes, permitindo criar campanhas de marketing direcionadas de acordo com o perfil dos usuários, ao mesmo tempo que fornece insights valiosos para a equipe de desenvolvimento.

Ao subir o servidor, é possível visualizar o dashboard em http://localhost:8050/


In [31]:
# 1. Receita por Área da Loja (Categoria)
area_sales = df_products.groupby('product_category')['product_revenue'].sum().reset_index()
area_sales = area_sales.sort_values(by='product_revenue', ascending=False)

fig1 = px.bar(area_sales, x='product_category', y='product_revenue', title='Receita Total por Área da Loja (Categoria)', labels={'product_category': 'Área da Loja (Categoria)', 'product_revenue': 'Receita Total'})

# 2. Top 10 Produtos por Receita
product_sales = df_products.groupby('product_name')['product_revenue'].sum().reset_index()
product_sales = product_sales.sort_values(by='product_revenue', ascending=False).head(10)

fig2 = px.bar(product_sales, x='product_revenue', y='product_name', orientation='h', title='Top 10 Produtos por Receita', labels={'product_name': 'Produto', 'product_revenue': 'Receita Total'})

# 3. Distribuição das Receitas dos Produtos
fig3 = px.histogram(df_products, x='product_revenue', nbins=50, title='Distribuição das Receitas dos Produtos', labels={'product_revenue': 'Receita do Produto', 'count': 'Frequência'})

# Análise de receita total por traffic_source, traffic_medium, traffic_campaign, device_category, device_os, device_browser, device_is_mobile
traffic_source_sales = df_products.groupby('traffic_source')['product_revenue'].sum().reset_index().sort_values(by='product_revenue', ascending=False)
fig4 = px.bar(traffic_source_sales, x='product_revenue', y='traffic_source', orientation='h', title='Receita Total por Fonte de Tráfego', labels={'traffic_source': 'Fonte de Tráfego', 'product_revenue': 'Receita Total'})

traffic_medium_sales = df_products[df_products['traffic_medium'] != '(none)'].groupby('traffic_medium')['product_revenue'].sum().reset_index().sort_values(by='product_revenue', ascending=False)
fig5 = px.bar(traffic_medium_sales, x='product_revenue', y='traffic_medium', orientation='h', title='Receita Total por Meio de Tráfego', labels={'traffic_medium': 'Meio de Tráfego', 'product_revenue': 'Receita Total'})

traffic_campaign_sales = df_products[df_products['traffic_campaign'] != '(not set)'].groupby('traffic_campaign')['product_revenue'].sum().reset_index().sort_values(by='product_revenue', ascending=False)
fig6 = px.bar(traffic_campaign_sales, x='product_revenue', y='traffic_campaign', orientation='h', title='Receita Total por Tráfego de Campanha', labels={'traffic_campaign': 'Tráfego de Campanha', 'product_revenue': 'Receita Total'})

device_category_sales = df_products.groupby('device_category')['product_revenue'].sum().reset_index().sort_values(by='product_revenue', ascending=False)
fig7 = px.bar(device_category_sales, x='product_revenue', y='device_category', orientation='h', title='Receita Total por Categoria de Dispositivo', labels={'device_category': 'Categoria de Dispositivo', 'product_revenue': 'Receita Total'})

device_os_sales = df_products.groupby('device_os')['product_revenue'].sum().reset_index().sort_values(by='product_revenue', ascending=False)
fig8 = px.bar(device_os_sales, x='product_revenue', y='device_os', orientation='h', title='Receita Total por Sistema Operacional do Dispositivo', labels={'device_os': 'Sistema Operacional do Dispositivo', 'product_revenue': 'Receita Total'})

device_browser_sales = df_products.groupby('device_browser')['product_revenue'].sum().reset_index().sort_values(by='product_revenue', ascending=False)
fig9 = px.bar(device_browser_sales, x='product_revenue', y='device_browser', orientation='h', title='Receita Total por Navegador do Dispositivo', labels={'device_browser': 'Navegador do Dispositivo', 'product_revenue': 'Receita Total'})

device_is_mobile_sales = df_products.groupby('device_is_mobile')['product_revenue'].sum().reset_index()
device_is_mobile_sales['device_is_mobile'] = device_is_mobile_sales['device_is_mobile'].map({True: 'Mobile', False: 'Desktop'})
device_is_mobile_sales = device_is_mobile_sales.sort_values(by='product_revenue', ascending=False)
fig10 = px.bar(device_is_mobile_sales, x='product_revenue', y='device_is_mobile', orientation='h', title='Receita Total por Tipo de Dispositivo', labels={'device_is_mobile': 'Tipo de Dispositivo', 'product_revenue': 'Receita Total'})

# Iniciar a aplicação Dash
app = dash.Dash(__name__)

# Layout da aplicação Dash
app.layout = html.Div([
    html.H1("Dashboard de Análise de Vendas"),
    dcc.Tabs([
        dcc.Tab(label='Receita por Categoria', children=[
            dcc.Graph(id='fig1', figure=fig1)
        ]),
        dcc.Tab(label='Top 10 Produtos', children=[
            dcc.Graph(id='fig2', figure=fig2)
        ]),
        dcc.Tab(label='Distribuição das Receitas', children=[
            dcc.Graph(id='fig3', figure=fig3)
        ]),
        dcc.Tab(label='Receita por Fonte de Tráfego', children=[
            dcc.Graph(id='fig4', figure=fig4)
        ]),
        dcc.Tab(label='Receita por Meio de Tráfego', children=[
            dcc.Graph(id='fig5', figure=fig5)
        ]),
        dcc.Tab(label='Receita por Tráfego de Campanha', children=[
            dcc.Graph(id='fig6', figure=fig6)
        ]),
        dcc.Tab(label='Receita por Categoria de Dispositivo', children=[
            dcc.Graph(id='fig7', figure=fig7)
        ]),
        dcc.Tab(label='Receita por Sistema Operacional', children=[
            dcc.Graph(id='fig8', figure=fig8)
        ]),
        dcc.Tab(label='Receita por Navegador', children=[
            dcc.Graph(id='fig9', figure=fig9)
        ]),
        dcc.Tab(label='Receita por Tipo de Dispositivo', children=[
            dcc.Graph(id='fig10', figure=fig10)
        ]),
        dcc.Tab(label='Mapa Interativo de Acessos e Compras', children=[
            html.Div([
                html.H2("Mapa Interativo de Acessos e Compras"),
                dcc.Checklist(
                    id='filter_purchases',
                    options=[{'label': 'Somente com Compras', 'value': 'purchases'}],
                    value=[]
                ),
                html.Iframe(id='map', srcDoc=open('mapa_acessos.html', 'r').read(), width='100%', height='900')
            ])
        ])
    ])
])

# Callback para atualizar o mapa com o filtro
@app.callback(
    Output('map', 'srcDoc'),
    Input('filter_purchases', 'value')
)

def update_map(filter_values):
    filtered_df = country_df
    if 'purchases' in filter_values:
        filtered_df = filtered_df[filtered_df['total_purchases'] > 0]

    map_access = folium.Map(location=[0, 0], zoom_start=2.5)
    heat_data = [[row['latitude'], row['longitude'], row['total_accesses']] for index, row in filtered_df.iterrows()]
    HeatMap(heat_data).add_to(map_access)
    map_access.save('filtered_map.html')
    
    return open('filtered_map.html', 'r').read()

# Executar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)
